In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans

In [2]:
# 주어진 데이터를 각각 불러옵니다.
weekday_clustered = pd.read_csv('data/processed/weekday_clusted.csv')
friday_clustered = pd.read_csv('data/processed/friday_clusted.csv')
weekend_clustered = pd.read_csv('data/processed/weekend_clusted.csv')

# 3. 좌표 추천하기

### 1. 1차 추천

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 각 요일에 해당하는 클러스터링 모델과 데이터셋을 미리 준비 (weekday_clustered, friday_clustered, weekend_clustered)

def choose_model_by_day(day):
    """요일에 따라 적절한 클러스터링 모델을 선택하는 함수"""
    if day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday']:
        return weekday_clustered
    elif day == 'Friday':
        return friday_clustered
    else:
        return weekend_clustered

random_state = 0



def get_cluster_for_input(model_data, input_coords, input_time, input_temp, input_rain, is_holiday):
    """
    입력된 좌표, 시간, 기온, 강수량, 공휴일 여부 등을 바탕으로 적절한 클러스터를 할당하는 함수
    KMeans 클러스터링을 이용해 클러스터 레이블을 예측합니다.
    """
    # 특성 선택: 클러스터링에 사용한 동일한 특성
    features = ['승차X좌표', '승차Y좌표', 'hour', '기온(°C)', '강수량(mm)', 'holiday']

    # 클러스터링을 수행한 데이터를 기준으로 표준화 스케일러 학습
    scaler = StandardScaler()
    X = model_data[features]
    scaled_X = scaler.fit_transform(X)

    # KMeans 모델 학습
    n_clusters = model_data['cluster_multi'].nunique()  # 기존 데이터의 클러스터 개수를 기반으로 설정
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(scaled_X)

    # 입력 데이터를 표준화하여 준비
    input_data = [[input_coords[0], input_coords[1], input_time, input_temp, input_rain, is_holiday]]
    scaled_input_data = scaler.transform(input_data)

    # 입력 데이터의 클러스터를 예측
    cluster_label = kmeans.predict(scaled_input_data)

    return cluster_label[0]  # 예측된 클러스터 레이블 반환



def recommend_top_locations(model_data, cluster_label, input_coords):
    """해당 클러스터 내에서 가장 빈번한 위치를 추천하는 함수"""
    # 해당 클러스터 데이터만 추출
    cluster_data = model_data[model_data['cluster_multi'] == cluster_label]

    # 좌표 빈도수를 계산해서 top 10을 추출
    top_locations = cluster_data[['승차X좌표', '승차Y좌표']].value_counts().head(15)

    return top_locations


def get_reason_for_recommendation(day, cluster_label):
    """추천 이유를 출력하는 함수"""
    if day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday']:
        if cluster_label == 0:
            return "평일 쌀쌀한 아침 시간대 승차가 많은 곳 리스트"
        elif cluster_label == 1:
            return "평일 비내리는 저녁 시간대 승차가 많은 곳 리스트"
        elif cluster_label == 2:
            return "평일 공휴일 낮시간대 승차가 많은 곳 리스트"
    elif day == 'Friday':
        if cluster_label == 0:
            return "금요일 저녁 흐린 시간대 승차가 많은 곳 리스트"
        elif cluster_label == 1:
            return "금요일 쌀쌀한 새벽 시간대 승차가 많은 곳 리스트"
        elif cluster_label == 2:
            return "금요일 공휴일 비내리는 정오 시간대 승차가 많은 곳 리스트"
    else:  # 주말인 경우
        if cluster_label == 0:
            return "주말 쌀쌀한 새벽 시간대 승차가 많은 곳 리스트"
        elif cluster_label == 1:
            return "주말 따뜻한 저녁 시간대 승차가 많은 곳 리스트"
        elif cluster_label == 2:
            return "주말 공휴일 추운 오후 시간대 승차가 많은 곳 리스트"


def recommend_locations(day, input_coords, input_time, input_temp, input_rain, is_holiday):
    """사용자에게 추천 위치를 제공하는 메인 함수"""
    # 요일에 맞는 클러스터링 모델 선택
    model_data = choose_model_by_day(day)

    # 현재 좌표, 시간, 기온, 강수량, 공휴일 여부로 클러스터 선택
    cluster_label = get_cluster_for_input(model_data, input_coords, input_time, input_temp, input_rain, is_holiday)

    # 추천 이유 출력
    reason = get_reason_for_recommendation(day, cluster_label)
    print(f"현재 상황과 유사한 승차 패턴을 추천합니다. \n{reason}")


    # 해당 클러스터에서 가장 많이 사용된 top 10 좌표 추천
    top_locations = recommend_top_locations(model_data, cluster_label, input_coords)

    return top_locations


In [4]:
### 공휴일 예시 1

In [5]:
day = 'Thursday'
current_coords = [127.388463, 36.342952]  # 예시 좌표 (경도, 위도)
current_time = 8
current_temp = 18  # 현재 기온 (예시)
current_rain = 0
is_holiday = 0

# 추천 위치 출력
top_10_locations = recommend_locations(day, current_coords, current_time, current_temp, current_rain, is_holiday)
print(top_10_locations)


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
평일 쌀쌀한 아침 시간대 승차가 많은 곳 리스트
승차X좌표     승차Y좌표  
127.4329  36.3319    399
127.3774  36.3517    278
127.3773  36.3517    276
127.3775  36.3517    197
127.4328  36.3319    186
127.3772  36.3518    175
127.3776  36.3499    139
127.3148  36.3918    103
127.3775  36.3498    102
127.4391  36.3650     95
127.4266  36.3274     94
127.4268  36.3271     91
127.3776  36.3516     87
127.4250  36.3266     85
127.3763  36.3502     83
Name: count, dtype: int64


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [6]:
day = 'Thursday'
current_coords = [127.388463, 36.342952]  # 예시 좌표 (경도, 위도)
current_time = 8
current_temp = 18  # 현재 기온 (예시)
current_rain = 0
is_holiday = 1

# 추천 위치 출력
top_10_locations = recommend_locations(day, current_coords, current_time, current_temp, current_rain, is_holiday)
print(top_10_locations)


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
평일 공휴일 낮시간대 승차가 많은 곳 리스트
승차X좌표     승차Y좌표  
127.4329  36.3319    70
127.4328  36.3319    51
127.4376  36.3502    38
127.4362  36.3324    32
127.4040  36.3225    31
127.4323  36.3319    30
127.3774  36.3517    25
127.4377  36.3501    18
127.3776  36.3499    18
127.3771  36.3518    17
127.3772  36.3518    16
127.4376  36.3503    16
127.4267  36.3272    15
127.4330  36.3319    15
127.4268  36.3271    15
Name: count, dtype: int64


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [7]:
### 공휴일 예시 2

In [8]:
day = 'Thursday'
current_coords = [127.388463, 36.342952	]  # 예시 좌표 (경도, 위도)
current_time = 5
current_temp = 23 # 현재 기온 (예시)
current_rain = 0
is_holiday = 1

# 추천 위치 출력
top_10_locations = recommend_locations(day, current_coords, current_time, current_temp, current_rain, is_holiday)
print(top_10_locations)

/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
평일 공휴일 낮시간대 승차가 많은 곳 리스트
승차X좌표     승차Y좌표  
127.4329  36.3319    70
127.4328  36.3319    51
127.4376  36.3502    38
127.4362  36.3324    32
127.4040  36.3225    31
127.4323  36.3319    30
127.3774  36.3517    25
127.4377  36.3501    18
127.3776  36.3499    18
127.3771  36.3518    17
127.3772  36.3518    16
127.4376  36.3503    16
127.4267  36.3272    15
127.4330  36.3319    15
127.4268  36.3271    15
Name: count, dtype: int64


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [9]:
day = 'Thursday'
current_coords = [127.388463, 36.342952	]  # 예시 좌표 (경도, 위도)
current_time = 5
current_temp = 23 # 현재 기온 (예시)
current_rain = 0
is_holiday = 0

# 추천 위치 출력
top_10_locations = recommend_locations(day, current_coords, current_time, current_temp, current_rain, is_holiday)
print(top_10_locations)

/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
평일 쌀쌀한 아침 시간대 승차가 많은 곳 리스트
승차X좌표     승차Y좌표  
127.4329  36.3319    399
127.3774  36.3517    278
127.3773  36.3517    276
127.3775  36.3517    197
127.4328  36.3319    186
127.3772  36.3518    175
127.3776  36.3499    139
127.3148  36.3918    103
127.3775  36.3498    102
127.4391  36.3650     95
127.4266  36.3274     94
127.4268  36.3271     91
127.3776  36.3516     87
127.4250  36.3266     85
127.3763  36.3502     83
Name: count, dtype: int64


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [10]:
### 주중 예시 

In [11]:
day = 'Thursday'
current_coords = [127.388463, 36.342952	]  # 예시 좌표 (경도, 위도)
current_time = 0
current_temp = 23 # 현재 기온 (예시)
current_rain = 0.1
is_holiday = 0

# 추천 위치 출력
top_10_locations = recommend_locations(day, current_coords, current_time, current_temp, current_rain, is_holiday)
print(top_10_locations)

/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
평일 쌀쌀한 아침 시간대 승차가 많은 곳 리스트
승차X좌표     승차Y좌표  
127.4329  36.3319    399
127.3774  36.3517    278
127.3773  36.3517    276
127.3775  36.3517    197
127.4328  36.3319    186
127.3772  36.3518    175
127.3776  36.3499    139
127.3148  36.3918    103
127.3775  36.3498    102
127.4391  36.3650     95
127.4266  36.3274     94
127.4268  36.3271     91
127.3776  36.3516     87
127.4250  36.3266     85
127.3763  36.3502     83
Name: count, dtype: int64


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [12]:
day = 'Thursday'
current_coords = [127.394437, 36.339408]  # 예시 좌표 (경도, 위도)
current_time = 16
current_temp = 3 # 현재 기온 (예시)
current_rain = 0
is_holiday = 0  # 공휴일 여부 (0: 비공휴일, 1: 공휴일)

# 추천 위치 출력
top_10_locations = recommend_locations(day, current_coords, current_time, current_temp, current_rain, is_holiday)
print(top_10_locations)

/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
평일 비내리는 저녁 시간대 승차가 많은 곳 리스트
승차X좌표     승차Y좌표  
127.4329  36.3319    1360
127.4328  36.3319     699
127.4362  36.3324     672
127.4361  36.3324     472
127.4040  36.3225     426
127.4376  36.3502     225
127.4323  36.3319     214
127.4330  36.3319     214
127.4040  36.3226     164
127.4377  36.3501     164
127.4369  36.3515     150
127.4321  36.3325     145
127.4268  36.3271     144
127.3775  36.3517     139
127.3772  36.3518     123
Name: count, dtype: int64


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


2. 금요일

In [13]:
day = 'Friday'
current_coords = [127.363753, 36.385640]  # 예시 좌표 (경도, 위도)
current_time = 17
current_temp = 22 # 현재 기온 (예시)
current_rain = 0.2
is_holiday = 1  # 공휴일 여부 (0: 비공휴일, 1: 공휴일)

# 추천 위치 출력
top_10_locations = recommend_locations(day, current_coords, current_time, current_temp, current_rain, is_holiday)
print(top_10_locations)

/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
금요일 공휴일 비내리는 정오 시간대 승차가 많은 곳 리스트
승차X좌표     승차Y좌표  
127.4329  36.3319    28
127.4376  36.3502    15
127.4361  36.3324    12
127.4362  36.3324    12
127.3776  36.3499    11
127.4376  36.3503    10
127.4040  36.3225     9
127.4277  36.3293     9
127.4328  36.3319     8
127.4365  36.3491     8
127.4267  36.3272     7
127.3775  36.3517     7
127.4266  36.3274     7
127.3775  36.3497     7
127.3771  36.3518     6
Name: count, dtype: int64


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [14]:
day = 'Friday'
current_coords = [127.387520, 36.343205]  # 예시 좌표 (경도, 위도)
current_time = 3
current_temp = 22 # 현재 기온 (예시)
current_rain = 0.8
is_holiday = 0  # 공휴일 여부 (0: 비공휴일, 1: 공휴일)

# 추천 위치 출력
top_10_locations = recommend_locations(day, current_coords, current_time, current_temp, current_rain, is_holiday)
print(top_10_locations)

/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
금요일 쌀쌀한 새벽 시간대 승차가 많은 곳 리스트
승차X좌표     승차Y좌표  
127.3773  36.3517    102
127.3774  36.3517     96
127.3772  36.3518     69
127.3775  36.3517     59
127.4329  36.3319     49
127.3776  36.3499     44
127.3775  36.3498     34
127.4328  36.3319     29
127.3881  36.3407     29
127.3776  36.3498     27
127.3763  36.3502     26
127.4105  36.3219     25
127.3776  36.3516     24
127.3448  36.3583     24
127.4250  36.3267     23
Name: count, dtype: int64


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [15]:
day = 'Friday'
current_coords = [127.406608, 36.335903]  # 예시 좌표 (경도, 위도)
current_time = 18
current_temp = 27 # 현재 기온 (예시)
current_rain = 0.2
is_holiday = 0 # 공휴일 여부 (0: 비공휴일, 1: 공휴일)

# 추천 위치 출력
top_10_locations = recommend_locations(day, current_coords, current_time, current_temp, current_rain, is_holiday)
print(top_10_locations)

/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
금요일 저녁 흐린 시간대 승차가 많은 곳 리스트
승차X좌표     승차Y좌표  
127.4329  36.3319    450
127.4328  36.3319    187
127.4362  36.3324    187
127.4040  36.3225    172
127.4361  36.3324    147
127.4376  36.3502    116
127.4323  36.3319     72
127.4330  36.3319     69
127.4376  36.3503     62
127.4040  36.3226     58
127.4369  36.3515     57
127.4377  36.3501     57
127.4268  36.3271     46
127.4321  36.3325     45
127.4369  36.3516     42
Name: count, dtype: int64


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


3. 주말

In [16]:
day = 'Sunday'
current_coords = [127.398481, 36.338375]  # 예시 좌표 (경도, 위도)
current_time = 18
current_temp = 3 # 현재 기온 (예시)
current_rain = 0
is_holiday = 1  # 공휴일 여부 (0: 비공휴일, 1: 공휴일)

# 추천 위치 출력
top_10_locations = recommend_locations(day, current_coords, current_time, current_temp, current_rain, is_holiday)
print(top_10_locations)

/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
주말 공휴일 추운 오후 시간대 승차가 많은 곳 리스트
승차X좌표     승차Y좌표  
127.4040  36.3225    6
127.4324  36.3317    6
          36.3318    5
127.3772  36.3518    5
127.4040  36.3226    5
127.4369  36.3515    5
127.3774  36.3517    5
127.4323  36.3319    5
127.4320  36.3326    4
127.3776  36.3499    4
127.3768  36.3519    4
127.4266  36.3274    4
127.4361  36.3324    4
127.4376  36.3502    4
127.4365  36.3524    3
Name: count, dtype: int64


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [17]:
day = 'Sunday'
current_coords = [127.388138, 36.342018]  # 예시 좌표 (경도, 위도)
current_time = 3
current_temp = 10 # 현재 기온 (예시)
current_rain = 0.03
is_holiday = 0  # 공휴일 여부 (0: 비공휴일, 1: 공휴일)

# 추천 위치 출력
top_10_locations = recommend_locations(day, current_coords, current_time, current_temp, current_rain, is_holiday)
print(top_10_locations)

/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
주말 쌀쌀한 새벽 시간대 승차가 많은 곳 리스트
승차X좌표     승차Y좌표  
127.3773  36.3517    271
127.3772  36.3518    262
127.3776  36.3499    201
127.3774  36.3517    191
127.3775  36.3517    159
127.4329  36.3319    142
127.4266  36.3274    134
127.3777  36.3500    124
127.3771  36.3518    123
127.3775  36.3498    113
127.3913  36.3148    109
127.4365  36.3524    102
127.3776  36.3516     95
          36.3498     91
127.4268  36.3271     88
Name: count, dtype: int64


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [18]:
###########################################################

### 2. 2차 좌표추천 (동점 가중치)

In [19]:
# 인기 관광지 및 택시 승강장 데이터를 불러오기
tourist_spots = pd.read_csv('data/extra/인기관광지_위도경도.csv')
terminal = pd.read_csv('data/extra/택시승강장_위도경도.csv')

In [20]:
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import pandas as pd

# 거리 계산 함수 (Haversine 거리 계산 함수)
def haversine_distance(coord1, coord2):
    """경도, 위도를 입력받아 두 점 사이의 Haversine 거리 계산"""
    lon1, lat1, lon2, lat2 = map(radians, [coord1[0], coord1[1], coord2[0], coord2[1]])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    radius = 6371  # 지구 반지름 (킬로미터)
    return radius * c

# 관광지 데이터에서 [경도, 위도] 형식으로 변환
popular_locations = list(zip(tourist_spots['경도'], tourist_spots['위도']))

# 택시 승강장 데이터에서 [경도, 위도] 형식으로 변환
taxi_stands = list(zip(terminal['경도'], terminal['위도']))

# 각 클러스터 내의 인기 관광지 및 택시 승강장 찾기 함수
def find_nearest_locations(cluster_center, location_list, threshold_distance=10):
    """클러스터의 중심점으로부터 가까운 위치들(관광지 또는 승강장)을 찾는 함수"""
    nearby_locations = []
    for loc in location_list:
        distance = haversine_distance(cluster_center, loc)
        if distance <= threshold_distance:  # 임계값 내에 있는 위치들만 포함
            nearby_locations.append(loc)
    return nearby_locations

# 빈도 동점 처리 및 관광지와 택시 승강장과의 거리로 재정렬
def rank_by_weighted_distance(top_locations, nearest_tourist_spots, nearest_taxi_stands, tourist_weight=0.7, taxi_weight=0.3):
    """동점인 위치들에 대해 관광지와 택시 승강장과의 가중치를 고려한 점수로 재정렬"""
    ranked_locations = []

    for location in top_locations.index:
        coord = location  # 좌표
        freq = top_locations[location]  # 빈도

        # 해당 위치와 관광지까지의 최소 거리 계산
        tourist_distances = [haversine_distance(coord, spot) for spot in nearest_tourist_spots] if nearest_tourist_spots else [100]
        min_tourist_distance = min(tourist_distances)

        # 해당 위치와 택시 승강장까지의 최소 거리 계산
        taxi_distances = [haversine_distance(coord, stand) for stand in nearest_taxi_stands] if nearest_taxi_stands else [100]
        min_taxi_distance = min(taxi_distances)

        # 최종 점수 계산: 가중치 적용
        weighted_score = (min_tourist_distance * tourist_weight) + (min_taxi_distance * taxi_weight)
        ranked_locations.append((coord, freq, weighted_score))

    # 빈도 우선 정렬, 동점일 때 가중치 적용 점수를 기준으로 재정렬
    ranked_locations.sort(key=lambda x: (-x[1], x[2]))  # 빈도 내림차순, 가중치 점수 오름차순

    return ranked_locations


 

from IPython.display import display
import folium


def recommend_locations_with_weights(day, input_coords, input_time, input_temp, input_rain, is_holiday, tourist_weight=0.7, taxi_weight=0.3):
    """사용자에게 클러스터 내 인기 관광지 및 택시 승강장을 고려한 추천 위치 제공"""
    # 요일에 맞는 클러스터링 모델 선택
    model_data = choose_model_by_day(day)

    # 현재 좌표, 시간, 기온, 강수량, 공휴일 여부로 클러스터 선택
    cluster_label = get_cluster_for_input(model_data, input_coords, input_time, input_temp, input_rain, is_holiday)

    # 추천 이유 출력
    reason = get_reason_for_recommendation(day, cluster_label)
    print(f"현재 상황과 유사한 승차 패턴을 추천합니다. \n{reason}")

    # 해당 클러스터에서 가장 많이 사용된 top 10 좌표 추천
    top_locations = recommend_top_locations(model_data, cluster_label, input_coords)

    # 클러스터 중심 계산 (클러스터 내 평균 좌표 사용)
    cluster_center = model_data[model_data['cluster_multi'] == cluster_label][['승차X좌표', '승차Y좌표']].mean().values

    # 클러스터 중심 좌표를 경도, 위도 순으로 변환
    cluster_center = [cluster_center[0], cluster_center[1]]

    # 해당 클러스터에 포함된 인기 관광지 및 택시 승강장 찾기
    nearest_tourist_spots = find_nearest_locations(cluster_center, popular_locations)
    nearest_taxi_stands = find_nearest_locations(cluster_center, taxi_stands)

    # 빈도 동점 처리: 관광지와 택시 승강장과 가까운 순서로 가중치 계산 후 재정렬
    ranked_locations = rank_by_weighted_distance(top_locations, nearest_tourist_spots, nearest_taxi_stands, tourist_weight, taxi_weight)
    # 상위 10개만 선택
    top_10_ranked_locations = ranked_locations[:10]

    # 출력 형식 수정
    for i, loc in enumerate(top_10_ranked_locations, start=1):
        print(f"추천 위치 {i}: 좌표: {loc[0]}, 빈도: {loc[1]}, 가중치 적용 점수: {loc[2]:.2f}")

    # 지도 생성 및 위치 마커 추가
    map_center = [input_coords[1], input_coords[0]]  # 중심을 입력 좌표로 설정
    m = folium.Map(location=map_center, zoom_start=14)  # 초기 zoom level 설정

    # 현재 좌표를 노란색 마커로 추가
    folium.Marker(
        location=[input_coords[1], input_coords[0]],  # 위도, 경도 순으로 위치 지정
        popup="현재 위치",
        icon=folium.Icon(color='yellow')
    ).add_to(m)

    # 추천 위치를 지도에 마커로 추가
    for i, loc in enumerate(top_10_ranked_locations, start=1):
        folium.Marker(
            location=[loc[0][1], loc[0][0]],  # 위도, 경도 순으로 위치 지정
            popup=f"추천 위치 {i}: 빈도: {loc[1]}, 가중치 점수: {loc[2]:.2f}",
            icon=folium.Icon(color='blue')
        ).add_to(m)

    # 지도 표시
    display(m)  # Jupyter Notebook에서 지도를 바로 표시
    return m  # m을 반환하여 지도 객체로도 사용할 수 있게 함



In [21]:
day = 'Thursday'
current_coords = [127.393018, 36.346918]  # 예시 좌표 (경도, 위도)
current_time = 16
current_temp = 17  # 현재 기온 (예시)
current_rain = 10
is_holiday = 0  # 공휴일 여부 (예: 0 = 공휴일 아님, 1 = 공휴일)

# 관광지 및 택시 승강장을 고려한 추천 위치 출력
# 관광지 가중치 0.8, 택시 승강장 가중치 0.2 예시
top_10_locations_with_weights = recommend_locations_with_weights(
    day, current_coords, current_time, current_temp, current_rain, is_holiday,
    tourist_weight=0.8, taxi_weight=0.2
)


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
평일 비내리는 저녁 시간대 승차가 많은 곳 리스트
추천 위치 1: 좌표: (127.4329, 36.3319), 빈도: 1360, 가중치 적용 점수: 0.13
추천 위치 2: 좌표: (127.4328, 36.3319), 빈도: 699, 가중치 적용 점수: 0.13
추천 위치 3: 좌표: (127.4362, 36.3324), 빈도: 672, 가중치 적용 점수: 0.33
추천 위치 4: 좌표: (127.4361, 36.3324), 빈도: 472, 가중치 적용 점수: 0.32
추천 위치 5: 좌표: (127.404, 36.3225), 빈도: 426, 가중치 적용 점수: 0.33
추천 위치 6: 좌표: (127.4376, 36.3502), 빈도: 225, 가중치 적용 점수: 0.10
추천 위치 7: 좌표: (127.433, 36.3319), 빈도: 214, 가중치 적용 점수: 0.14
추천 위치 8: 좌표: (127.4323, 36.3319), 빈도: 214, 가중치 적용 점수: 0.14
추천 위치 9: 좌표: (127.4377, 36.3501), 빈도: 164, 가중치 적용 점수: 0.11
추천 위치 10: 좌표: (127.404, 36.3226), 빈도: 164, 가중치 적용 점수: 0.32


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/var/folders/tg/ntvm9wc94h937xqtzftck9xc0000gn/T/ipykernel_30099/406065404.py:108: UserWarning: color argument of Icon should be one of: {'cadetblue', 'purple', 'darkgreen', 'beige', 'gray', 'orange', 'darkred', 'lightblue', 'lightgray', 'darkblue', 'white', 'green', 'black', 'pink', 'blue', 'red', 'lightgreen', 'darkpurple', 'lightred'}.
  icon=folium.Icon(color='yellow')


In [22]:
# 사용 예시
day = 'Thursday'
current_coords = [127.393018, 36.346918]  # 예시 좌표 (경도, 위도)
current_time = 16
current_temp = 17  # 현재 기온 (예시)
current_rain = 10
is_holiday = 1  # 공휴일 여부 (예: 0 = 공휴일 아님, 1 = 공휴일)

# 관광지 및 택시 승강장을 고려한 추천 위치 출력
# 관광지 가중치 0.8, 택시 승강장 가중치 0.2 예시
top_10_locations_with_weights = recommend_locations_with_weights(
    day, current_coords, current_time, current_temp, current_rain, is_holiday,
    tourist_weight=0.8, taxi_weight=0.2
)


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
평일 공휴일 낮시간대 승차가 많은 곳 리스트
추천 위치 1: 좌표: (127.4329, 36.3319), 빈도: 70, 가중치 적용 점수: 0.13
추천 위치 2: 좌표: (127.4328, 36.3319), 빈도: 51, 가중치 적용 점수: 0.13
추천 위치 3: 좌표: (127.4376, 36.3502), 빈도: 38, 가중치 적용 점수: 0.10
추천 위치 4: 좌표: (127.4362, 36.3324), 빈도: 32, 가중치 적용 점수: 0.33
추천 위치 5: 좌표: (127.404, 36.3225), 빈도: 31, 가중치 적용 점수: 0.33
추천 위치 6: 좌표: (127.4323, 36.3319), 빈도: 30, 가중치 적용 점수: 0.14
추천 위치 7: 좌표: (127.3774, 36.3517), 빈도: 25, 가중치 적용 점수: 0.13
추천 위치 8: 좌표: (127.4377, 36.3501), 빈도: 18, 가중치 적용 점수: 0.11
추천 위치 9: 좌표: (127.3776, 36.3499), 빈도: 18, 가중치 적용 점수: 0.28
추천 위치 10: 좌표: (127.3771, 36.3518), 빈도: 17, 가중치 적용 점수: 0.14


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/var/folders/tg/ntvm9wc94h937xqtzftck9xc0000gn/T/ipykernel_30099/406065404.py:108: UserWarning: color argument of Icon should be one of: {'cadetblue', 'purple', 'darkgreen', 'beige', 'gray', 'orange', 'darkred', 'lightblue', 'lightgray', 'darkblue', 'white', 'green', 'black', 'pink', 'blue', 'red', 'lightgreen', 'darkpurple', 'lightred'}.
  icon=folium.Icon(color='yellow')


In [23]:
# 사용 예시
day = 'Friday'
current_coords = [127.394970, 36.344142]  # 예시 좌표 (경도, 위도)
current_time = 10
current_temp = 22 # 현재 기온 (예시)
current_rain = 8
is_holiday = 0 
# 관광지 및 택시 승강장을 고려한 추천 위치 출력
# 관광지 가중치 0.8, 택시 승강장 가중치 0.2 예시
top_10_locations_with_weights = recommend_locations_with_weights(
    day, current_coords, current_time, current_temp, current_rain, is_holiday,
    tourist_weight=0.8, taxi_weight=0.2
)


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
금요일 저녁 흐린 시간대 승차가 많은 곳 리스트
추천 위치 1: 좌표: (127.4329, 36.3319), 빈도: 450, 가중치 적용 점수: 0.13
추천 위치 2: 좌표: (127.4328, 36.3319), 빈도: 187, 가중치 적용 점수: 0.13
추천 위치 3: 좌표: (127.4362, 36.3324), 빈도: 187, 가중치 적용 점수: 0.33
추천 위치 4: 좌표: (127.404, 36.3225), 빈도: 172, 가중치 적용 점수: 0.33
추천 위치 5: 좌표: (127.4361, 36.3324), 빈도: 147, 가중치 적용 점수: 0.32
추천 위치 6: 좌표: (127.4376, 36.3502), 빈도: 116, 가중치 적용 점수: 0.10
추천 위치 7: 좌표: (127.4323, 36.3319), 빈도: 72, 가중치 적용 점수: 0.14
추천 위치 8: 좌표: (127.433, 36.3319), 빈도: 69, 가중치 적용 점수: 0.14
추천 위치 9: 좌표: (127.4376, 36.3503), 빈도: 62, 가중치 적용 점수: 0.10
추천 위치 10: 좌표: (127.404, 36.3226), 빈도: 58, 가중치 적용 점수: 0.32


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/var/folders/tg/ntvm9wc94h937xqtzftck9xc0000gn/T/ipykernel_30099/406065404.py:108: UserWarning: color argument of Icon should be one of: {'cadetblue', 'purple', 'darkgreen', 'beige', 'gray', 'orange', 'darkred', 'lightblue', 'lightgray', 'darkblue', 'white', 'green', 'black', 'pink', 'blue', 'red', 'lightgreen', 'darkpurple', 'lightred'}.
  icon=folium.Icon(color='yellow')


In [24]:
day = 'Sunday'
current_coords = [127.394472, 36.338375]  # 예시 좌표 (경도, 위도)
current_time = 18
current_temp = 3 # 현재 기온 (예시)
current_rain = 15
is_holiday = 0 
# 관광지 및 택시 승강장을 고려한 추천 위치 출력
# 관광지 가중치 0.8, 택시 승강장 가중치 0.2 예시
top_10_locations_with_weights = recommend_locations_with_weights(
    day, current_coords, current_time, current_temp, current_rain, is_holiday,
    tourist_weight=0.8, taxi_weight=0.2
)


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
주말 따뜻한 저녁 시간대 승차가 많은 곳 리스트
추천 위치 1: 좌표: (127.4329, 36.3319), 빈도: 1036, 가중치 적용 점수: 0.13
추천 위치 2: 좌표: (127.4362, 36.3324), 빈도: 521, 가중치 적용 점수: 0.33
추천 위치 3: 좌표: (127.4328, 36.3319), 빈도: 483, 가중치 적용 점수: 0.13
추천 위치 4: 좌표: (127.404, 36.3225), 빈도: 373, 가중치 적용 점수: 0.33
추천 위치 5: 좌표: (127.4376, 36.3502), 빈도: 370, 가중치 적용 점수: 0.10
추천 위치 6: 좌표: (127.4361, 36.3324), 빈도: 357, 가중치 적용 점수: 0.32
추천 위치 7: 좌표: (127.4323, 36.3319), 빈도: 258, 가중치 적용 점수: 0.14
추천 위치 8: 좌표: (127.4369, 36.3515), 빈도: 221, 가중치 적용 점수: 0.12
추천 위치 9: 좌표: (127.404, 36.3226), 빈도: 187, 가중치 적용 점수: 0.32
추천 위치 10: 좌표: (127.4376, 36.3503), 빈도: 178, 가중치 적용 점수: 0.10


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/var/folders/tg/ntvm9wc94h937xqtzftck9xc0000gn/T/ipykernel_30099/406065404.py:108: UserWarning: color argument of Icon should be one of: {'cadetblue', 'purple', 'darkgreen', 'beige', 'gray', 'orange', 'darkred', 'lightblue', 'lightgray', 'darkblue', 'white', 'green', 'black', 'pink', 'blue', 'red', 'lightgreen', 'darkpurple', 'lightred'}.
  icon=folium.Icon(color='yellow')


In [25]:
# 예시 테스트
#24.1.25
day = 'Thursday'
current_coords = [127.4333556,36.3321424]  # 대전역 광장
current_time = 8
current_temp = -9.7 # 현재 기온 (예시)
current_rain = 0
is_holiday = 0 
# 관광지 및 택시 승강장을 고려한 추천 위치 출력
# 관광지 가중치 0.8, 택시 승강장 가중치 0.2 예시
top_10_locations_with_weights = recommend_locations_with_weights(
    day, current_coords, current_time, current_temp, current_rain, is_holiday,
    tourist_weight=0.8, taxi_weight=0.2
)


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
평일 쌀쌀한 아침 시간대 승차가 많은 곳 리스트
추천 위치 1: 좌표: (127.4329, 36.3319), 빈도: 399, 가중치 적용 점수: 0.13
추천 위치 2: 좌표: (127.3774, 36.3517), 빈도: 278, 가중치 적용 점수: 0.13
추천 위치 3: 좌표: (127.3773, 36.3517), 빈도: 276, 가중치 적용 점수: 0.13
추천 위치 4: 좌표: (127.3775, 36.3517), 빈도: 197, 가중치 적용 점수: 0.12
추천 위치 5: 좌표: (127.4328, 36.3319), 빈도: 186, 가중치 적용 점수: 0.13
추천 위치 6: 좌표: (127.3772, 36.3518), 빈도: 175, 가중치 적용 점수: 0.13
추천 위치 7: 좌표: (127.3776, 36.3499), 빈도: 139, 가중치 적용 점수: 0.28
추천 위치 8: 좌표: (127.3148, 36.3918), 빈도: 103, 가중치 적용 점수: 2.44
추천 위치 9: 좌표: (127.3775, 36.3498), 빈도: 102, 가중치 적용 점수: 0.30
추천 위치 10: 좌표: (127.4391, 36.365), 빈도: 95, 가중치 적용 점수: 1.32


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/var/folders/tg/ntvm9wc94h937xqtzftck9xc0000gn/T/ipykernel_30099/406065404.py:108: UserWarning: color argument of Icon should be one of: {'cadetblue', 'purple', 'darkgreen', 'beige', 'gray', 'orange', 'darkred', 'lightblue', 'lightgray', 'darkblue', 'white', 'green', 'black', 'pink', 'blue', 'red', 'lightgreen', 'darkpurple', 'lightred'}.
  icon=folium.Icon(color='yellow')


In [26]:
day = 'Sunday'
current_coords = [127.4333556,36.3321424]  # 대전역 광장
current_time = 19
current_temp = 19.3 # 현재 기온 (예시)
current_rain = 0.6
is_holiday = 0 
# 관광지 및 택시 승강장을 고려한 추천 위치 출력
# 관광지 가중치 0.8, 택시 승강장 가중치 0.2 예시
top_10_locations_with_weights = recommend_locations_with_weights(
    day, current_coords, current_time, current_temp, current_rain, is_holiday,
    tourist_weight=0.8, taxi_weight=0.2
)


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
주말 따뜻한 저녁 시간대 승차가 많은 곳 리스트
추천 위치 1: 좌표: (127.4329, 36.3319), 빈도: 1036, 가중치 적용 점수: 0.13
추천 위치 2: 좌표: (127.4362, 36.3324), 빈도: 521, 가중치 적용 점수: 0.33
추천 위치 3: 좌표: (127.4328, 36.3319), 빈도: 483, 가중치 적용 점수: 0.13
추천 위치 4: 좌표: (127.404, 36.3225), 빈도: 373, 가중치 적용 점수: 0.33
추천 위치 5: 좌표: (127.4376, 36.3502), 빈도: 370, 가중치 적용 점수: 0.10
추천 위치 6: 좌표: (127.4361, 36.3324), 빈도: 357, 가중치 적용 점수: 0.32
추천 위치 7: 좌표: (127.4323, 36.3319), 빈도: 258, 가중치 적용 점수: 0.14
추천 위치 8: 좌표: (127.4369, 36.3515), 빈도: 221, 가중치 적용 점수: 0.12
추천 위치 9: 좌표: (127.404, 36.3226), 빈도: 187, 가중치 적용 점수: 0.32
추천 위치 10: 좌표: (127.4376, 36.3503), 빈도: 178, 가중치 적용 점수: 0.10


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/var/folders/tg/ntvm9wc94h937xqtzftck9xc0000gn/T/ipykernel_30099/406065404.py:108: UserWarning: color argument of Icon should be one of: {'cadetblue', 'purple', 'darkgreen', 'beige', 'gray', 'orange', 'darkred', 'lightblue', 'lightgray', 'darkblue', 'white', 'green', 'black', 'pink', 'blue', 'red', 'lightgreen', 'darkpurple', 'lightred'}.
  icon=folium.Icon(color='yellow')


In [27]:
day = 'Sunday'
current_coords = [127.4333556,36.3321424]  # 대전역 광장
current_time = 7
current_temp = -2 # 현재 기온 (예시)
current_rain = 0
is_holiday = 0 
# 관광지 및 택시 승강장을 고려한 추천 위치 출력
# 관광지 가중치 0.8, 택시 승강장 가중치 0.2 예시
top_10_locations_with_weights = recommend_locations_with_weights(
    day, current_coords, current_time, current_temp, current_rain, is_holiday,
    tourist_weight=0.8, taxi_weight=0.2
)


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


현재 상황과 유사한 승차 패턴을 추천합니다. 
주말 쌀쌀한 새벽 시간대 승차가 많은 곳 리스트
추천 위치 1: 좌표: (127.3773, 36.3517), 빈도: 271, 가중치 적용 점수: 0.13
추천 위치 2: 좌표: (127.3772, 36.3518), 빈도: 262, 가중치 적용 점수: 0.13
추천 위치 3: 좌표: (127.3776, 36.3499), 빈도: 201, 가중치 적용 점수: 0.28
추천 위치 4: 좌표: (127.3774, 36.3517), 빈도: 191, 가중치 적용 점수: 0.13
추천 위치 5: 좌표: (127.3775, 36.3517), 빈도: 159, 가중치 적용 점수: 0.12
추천 위치 6: 좌표: (127.4329, 36.3319), 빈도: 142, 가중치 적용 점수: 0.13
추천 위치 7: 좌표: (127.4266, 36.3274), 빈도: 134, 가중치 적용 점수: 0.11
추천 위치 8: 좌표: (127.3777, 36.35), 빈도: 124, 가중치 적용 점수: 0.27
추천 위치 9: 좌표: (127.3771, 36.3518), 빈도: 123, 가중치 적용 점수: 0.14
추천 위치 10: 좌표: (127.3775, 36.3498), 빈도: 113, 가중치 적용 점수: 0.30


/Users/nayeong-eun/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/var/folders/tg/ntvm9wc94h937xqtzftck9xc0000gn/T/ipykernel_30099/406065404.py:108: UserWarning: color argument of Icon should be one of: {'cadetblue', 'purple', 'darkgreen', 'beige', 'gray', 'orange', 'darkred', 'lightblue', 'lightgray', 'darkblue', 'white', 'green', 'black', 'pink', 'blue', 'red', 'lightgreen', 'darkpurple', 'lightred'}.
  icon=folium.Icon(color='yellow')
